#**Preparacion del entorno**

## **Instalacion de librerias**

In [ ]:
# Bibliotecas básicas que se usan en el experimento
import pandas as pd
import random
import numpy as np
import time
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

#imblearn, de clase desequilibrada,
!pip install -U imbalanced-learn
from imblearn.over_sampling import RandomOverSampler, SMOTE, BorderlineSMOTE,SVMSMOTE, ADASYN #in the case of this experiment, no use kmeanssmote

#Liberia sklearn 
from sklearn.impute import SimpleImputer # Univariate imputer for completing missing values with simple strategies.
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, GridSearchCV, RandomizedSearchCV #defines the data partitioning and cross validation strategy. In this experiment use GridsearhCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler #In this expermiento, is used StandardScaler
from sklearn.pipeline import Pipeline

#Metodos de seleccion de caracteristicas
from sklearn.feature_selection import SelectKBest, f_classif, chi2, mutual_info_classif, GenericUnivariateSelect, VarianceThreshold, SequentialFeatureSelector, RFE, SelectFromModel, RFECV #In this expermiento, is used SelectKBest with f_classif and mutual_info_classif
!pip install skfeature-chappers
from skfeature.function.similarity_based import fisher_score, SPEC, lap_score, reliefF, trace_ratio # https://jundongl.github.io/scikit-feature/html/skfeature.function.html
from skfeature.function.information_theoretical_based import CIFE, CMIM, MRMR, MIM, JMI
from skfeature.function.statistical_based import t_score, gini_index, chi_square, f_score, low_variance
from scipy.stats import wilcoxon #This function is used in the feature selection method, according to the Wilcoxon criteria.
!pip install Boruta
from boruta import BorutaPy

# clasificadores
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC, NuSVC, SVC
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.dummy import DummyClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF,DotProduct,Matern,RationalQuadratic,WhiteKernel
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

#metricas
from sklearn.metrics import f1_score,precision_score,recall_score,accuracy_score,confusion_matrix,roc_auc_score, classification_report

#Guardar el dataframe y la serie en google DRIVE
from google.colab import drive, files, auth
from google.auth import default
drive.mount('/content/drive')


##**Funciones**

#####**Funcion optimiza los hiperparametros RL, BI, MVS, PMC, PG, ADL**

In [ ]:
def RL_BI_MVS_PMC_PG_ADL():
  #Selection molecular subtype and hormone receptor
  problemas_elegidos = ["luminal_A","luminal_B","her2","triple_negativo"]
  
  numeración_clasificadores = {
                      "1":"LR", 
                      "2":"BI", 
                      "3":"MVS", 
                      "4":"PMC",
                      "5":"PG",
                      "7":"ADL"}
  print(numeración_clasificadores) 
  clasificador = numeración_clasificadores[input("ndique el valor del modelo de clasificación: ")]

  #hiperparametros de los clasificadores
  hiperparámetros_ = {
      "RL" : {"C":(list(np.logspace(-4, 4, 20))),
              "penalty":["l1","l2","none"],
              "solver" : ['lbfgs', 'liblinear']}, 
      "BI" : {'var_smoothing': np.logspace(11,-11, num=1000)}, 
      "MSV" : {'C':np.logspace(-6, 6, 13, base = 2.0),
               "gamma":["auto"], "probability" : [True]},
      "PMC": {
          "hidden_layer_sizes": [(8, 16, 32), (8, 16, 32)], 
          "max_iter" :[1000]},
      "PG": {"n_restarts_optimizer" : np.arange(1, 11)},
      "ADL": {'shrinkage': [None,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,'auto'],
              'n_components': [None] + [1, 2, 5, 8, 13, 21, 34, 55]}  
      }
  hiperparámetros = hiperparámetros_[clasificador]

  #ramdom_state
  Random_State = [0,42,50,60,90] 

   #Metodo_de_seleccion_de_características
  fsel = ['_IMC_', '_MIM_', '_IG_', '_MF_', '_IM_', '_RELF_', '_MRMR_', 'PT', 'PW', 'PF']
  
  #numero de caracteristicas seleccionadas
  n_caracteristicas = [1,2,4,8,16,32]


  return problemas_elegidos,clasificador,hiperparámetros,Random_State,fsel,n_caracteristicas

#####**Funcion optimiza los hiperparametros WMC, BA, EA, AD**

In [ ]:
def VMC_BA_EA_AD():
  #Seleccion del subtipo molecular
  problema_de_clasificacion = {"0":"luminal_A", 
                        "1":"luminal_B",
                        "2":"her2",
                        "3":"triple_negativo",
}
  print(problema_de_clasificacion)
  problemas_elegidos = problema_de_clasificacion[input("Indique el valor del subtipo molecular: ")] 
  
  #Clasificadores
  numeración_clasificadores = {"1":"VMC",
                      "2":"BA",
                      "3":"EA",
                      "4":"AD"}
  print(numeración_clasificadores) 
  clasificador = numeración_clasificadores[input("ndique el valor del modelo de clasificación: ")]

  #hiperparametros de los clasificadores
  hiperparámetros_ = {
      "VMC" : {"n_neighbors" : list(range(1, 500)),
               "metric": ["euclidean","manhattan", "minkowski"]},
      "BA" : {"n_estimators": list(range(1,1000,50))},
      "EA" : {"n_estimators": list(range(1,1000,50))},
      "AD": {'max_depth': [10,30,50,70,100,300,500,700,1000,3000,5000,7000, None],
             'min_samples_split': [2,5,10,20,50,100],
             'min_samples_leaf':[2,5,10,20,50,100]},
      }
  hiperparámetros = hiperparámetros_[clasificador]
 
  #ramdom_state
  Random_State_ = {"0":0,"42":42, "50":50, "60":60, "90":90} #Será necesario tantas veces? . Este artículo habla de cómo 5 es suficiente. https://www.sciencedirect.com/science/article/pii/S0957417421013622?via%3Dihub#tbl1
  print(list(Random_State_.keys()))
  Random_State = Random_State_[input("Indique el valor del random state: ")] 

  #Metodo_de_seleccion_de_características
  fsel = ['_IMC_', '_MIM_', '_IG_', '_MF_', '_IM_', '_RELF_', '_MRMR_', 'PT', 'PW', 'PF']
  
  #numero de caracteristicas seleccionadas
  n_caracteristicas = [1,2,4,8,16,32]


  return problemas_elegidos,clasificador,hiperparámetros,Random_State,fsel,n_caracteristicas



#####**Funcion optimiza los hiperparametros RA y RG**

In [ ]:
def RA_RG():
  #Seleccion del subtipo molecular
  problema_de_clasificacion = {"0":"luminal_A", 
                        "1":"luminal_B",
                        "2":"her2",
                        "3":"triple_negativo",
}
  print(problema_de_clasificacion)
  problemas_elegidos = problema_de_clasificacion[input("Indique el valor del subtipo molecular: ")] 
  
  #Clasificadores
  numeración_clasificadores = {"1":"RA",
                      "2":"RG"}
  print(numeración_clasificadores) 
  clasificador = numeración_clasificadores[input("ndique el valor del modelo de clasificación: ")]

  #Hiperparametros 
  hiperparámetros_ = {
      "RA" : {"n_estimators": list(range(1,1000,50)),
              'learning_rate':list(np.linspace(0.1, 2.0, 20, endpoint=True))},
      "RG": {"n_estimators": list(range(1,1000,50)),
              'learning_rate':list(np.linspace(0.1, 2.0, 20, endpoint=True))},
      }
  hiperparámetros = hiperparámetros_[clasificador]
  
  #ramdom_state
  Random_State_ = {"0":0,"42":42, "50":50, "60":60, "90":90} #Será necesario tantas veces? . Este artículo habla de cómo 5 es suficiente. https://www.sciencedirect.com/science/article/pii/S0957417421013622?via%3Dihub#tbl1
  print(list(Random_State_.keys()))
  Random_State = Random_State_[input("Indique el valor del random state: ")] 

   #Metodo_de_seleccion_de_características
  Metodo_de_seleccion_de_características = { "1":"_IMC_",
                                      "2":"_MIM_",
                                      "3":"_IG_",
                                      "4":"_MF_",
                                      "5":"_IM_",
                                      "6": "_RELF_",
                                      "7": "_MRMR_",
                                      "8": "_PT_",
                                      "9": "_PW_",
                                      "10":"_PF_"
                                  }
  print(Metodo_de_seleccion_de_características)
  fsel = Metodo_de_seleccion_de_características[input("Indique el valor del método de selección de características:")]
  
  #numero de caracteristicas seleccionadas
  n_caracteristicas = [1,2,4,8,16,32]

  return problemas_elegidos,clasificador,hiperparámetros,Random_State,fsel,n_caracteristicas

##**Gridsearch**

In [ ]:
#iterativo 
def gridsearch_(problemas_elegidos,clasificador,hiperparámetros,Random_State,fsel,n_caracteristicas):

 #Validador cruzado K-Fold estratificado repetido.
  CV =  RepeatedStratifiedKFold(n_splits = 10, n_repeats = 1, random_state = 42)

  #clasificadores
  clasificadores = {"VMC" : KNeighborsClassifier(),
                    "RL" : LogisticRegression(),
                    "BI" : GaussianNB(),
                    "MVS" : SVC(),
                    "BA" : RandomForestClassifier(),
                    "RA" : AdaBoostClassifier(),
                    "EA" : ExtraTreesClassifier(),
                    "AD" : DecisionTreeClassifier(),
                    "RG" : GradientBoostingClassifier(),
                    "PMC" : MLPClassifier(),
                    "PG" : GaussianProcessClassifier(),
                    "ADL": LinearDiscriminantAnalysis()}
  

  #Estimador
  estimador_base = clasificadores[clasificador]
  nombre_estimador = str(estimador_base).replace('()',  '', 1) 

  
  #Exportar el conjunto de datos preprocesados
  X_train_fs.to_csv("/content/drive/MyDrive/tesis/"+ problemas_elegidos +"/seleccion_caracteristicas/"+fsel+"/_"+str(Random_State)+"/X_train/"+chosen_problems +"_rs_"+str(Random_State)+"_X_train"+fsel+str(n_caracteristicas)+".csv")
  X_train = X_train_fs_.drop(["Unnamed: 0"], axis = 1)
  y_train.to_csv("/content/drive/MyDrive/tesis/"+ problemas_elegidos +"/seleccion_caracteristicas/"+fsel+"/_"+str(Random_State)+"/y_train/"+chosen_problems +"_rs_"+str(Random_State)+"_y_train"+fsel+str(n_caracteristicas)+".csv")
  y_train = y_train_.drop(["Unnamed: 0"], axis = 1)
  X_test_fs.to_csv("/content/drive/MyDrive/tesis/"+ problemas_elegidos +"/seleccion_caracteristicas/"+fsel+"/_"+str(Random_State)+"/X_test/"+chosen_problems +"_rs_"+str(Random_State)+"_X_test"+fsel+str(n_caracteristicas)+".csv")
  X_test = X_test_fs_.drop(["Unnamed: 0"], axis = 1)
  y_test.to_csv("/content/drive/MyDrive/tesis/"+ problemas_elegidos +"/seleccion_caracteristicas/"+fsel+"/_"+str(Random_State)+"/y_test/"+chosen_problems +"_rs_"+str(Random_State)+"_y_test"+fsel+str(n_caracteristicas)+".csv")
  y_test = y_test_.drop(["Unnamed: 0"], axis = 1)
  

  #GridSearch
  start = time.time()  # Inicio del tiempo de ejecucion de la funcion GridSearchCV
  clf = GridSearchCV(base_estimator, param_grid= hyperparameters, cv = CV, n_jobs = -1, scoring='roc_auc').fit(X_train, y_train)
  end = time.time() # fin del tiempo de ejecucion de la funcion GridSearchCV
  times_clf = str(end - start) #times_clf, corresponde al tiempo de ejecución del método de selección de características
  roc_auc_train = clf.best_score_
  y_pred_1 = clf.predict(X_test)
  y_pred_2 = clf.predict_proba(X_test)[:, 1]
  best_estimator_gs = clf.best_estimator_
  best_params_gs = clf.best_params_
  
  #Puntuación del estimador
  roc_auc_test_1 = (roc_auc_score(y_test,y_pred_1))
  roc_auc_test_2 = (roc_auc_score(y_test,y_pred_2))
  f1_score_test_1 = (f1_score(y_test,y_pred_1))
  precision_score_test_1 = (precision_score(y_test,y_pred_1))
  recall_score_test_1 = (recall_score(y_test,y_pred_1))
  accuracy_score_test_1 = (accuracy_score(y_test,y_pred_1))

  #nombre de índice del dataframe de datos
  column_ = (chosen_problems + "_"+ classifier +"_gs_"+ str(Random_State)+ fsel + str(n_caracteristicas))
  #dataframe
  df_gs_ = pd.Series({
               "roc_train": roc_auc_train,
                "roc_test_1": roc_auc_test_1, 
                "roc_test_2": roc_auc_test_2,
                "f1_test_1":f1_score_test_1,
                "precision_test_1":precision_score_test_1,
                "recall_test_1":recall_score_test_1,
                "accuracy_test_1":accuracy_score_test_1,
                "time_clf_gs": times_clf,
                "estimator": best_estimator_gs}, name = column_ )
  df_gs_best_parameters =pd.Series(data = best_params_gs, name = column_)
  df_gs = pd.concat([df_gs_, df_gs_best_parameters])


  df_gs.to_csv("/content/drive/MyDrive/tesis/"+ problemas_elegidos +"/clasificadores/"+nombre_estimador+"/"+fsel+"/_"+str(Random_State)+"/"+problemas_elegidos +"_"+clasificador+"_rs_"+str(Random_State)+fsel+str(n_caracteristicas)+".csv")

#**Aplicacion del experimento**

##**RA and RG**

In [ ]:
#selecciona los parametros de la funcion gridsearch
problemas_elegidos,clasificador,hiperparámetros,Random_State,fsel,n_caracteristicas = RA_RG()

In [ ]:
#ejecucion del experimento
for i in range(len(n_feature)):
  gridsearch_(problemas_elegidos,clasificador,hiperparámetros,Random_State,fsel,n_caracteristicas[i])

##**VMC ,BA ,EA y AD**

In [ ]:
#selecciona los parametros de la funcion gridsearch
problemas_elegidos,clasificador,hiperparámetros,Random_State,fsel,n_caracteristicas = VMC_BA_EA_AD()

In [ ]:
#ejecucion del experimento
for x in range(len(fsel)):
  for i in range(len(n_feature)):
    gridsearch_(problemas_elegidos,clasificador,hiperparámetros,Random_State,fsel,n_caracteristicas[i])

##**RL ,BI ,VMC ,PMC ,PG y ADL**

In [ ]:
#selecciona los parametros de la funcion gridsearch
problemas_elegidos,clasificador,hiperparámetros,Random_State,fsel,n_caracteristicas = RL_BI_MVS_PMC_PG_ADL()

In [ ]:
#ejecucion del experimento
for z in range(4)):
  for x in range(10):
    for i in range(6):
      gridsearch_(problemas_elegidos,clasificador,hiperparámetros,Random_State,fsel,n_caracteristicas[i])